In [61]:
!pip install psycopg2
import psycopg2 as ps
import pandas as pd

In [69]:
# upload your CSV files here
df = pd.read_csv('spotify_track_data.csv', index_col=0)
df

,track_id,track_title,track_artists,track_album,track_release_date,energy,loudness,tempo,duration
0,3RBXNeUbe3X7ZBch00D2St,Extraordinary Girl / Letterbomb,Green Day,American Idiot,2004-09-21,0.958,-4.051,137.921,7:40
1,2gTdDMpNxIRFSiu7HutMCg,I Think I'm OKAY (with YUNGBLUD & Travis Barker),"Machine Gun Kelly, YUNGBLUD, Travis Barker",Hotel Diablo,2019-07-05,0.744,-4.718,119.921,2:49
2,6KnkYUz3zXQnftT1fzZcDL,다시 해줘 Do it again,TWICE,THE STORY BEGINS,2015-10-20,0.936,-0.948,136.983,3:26
3,0dy6iXYIF0piirySAzCBwF,The End Has No End,The Strokes,Room On Fire,2003-10-28,0.660,-3.725,136.027,3:05
4,7yCPwWs66K8Ba5lFuU2bcx,All The Small Things,blink-182,Enema Of The State,1999-01-01,0.891,-4.764,148.599,2:48
...,...,...,...,...,...,...,...,...,...
928,0ttHpMs250dpFkCYAnuhnE,Good,Better Than Ezra,Deluxe,1995-02-28,0.657,-6.873,109.950,3:05
929,13NCxLOlvQ4Tnexgfp03Gs,Do Me a Favour,Arctic Monkeys,Favourite Worst Nightmare (Standard Version),2007-04-24,0.735,-6.685,180.127,3:25
930,6tCssnvTUREcperDOUTqvA,WANNABE,ITZY,It'z Me,2020-03-09,0.911,-4.410,122.035,3:11
931,3yHQKddM8SVCRnuPSo3HPN,BOOMBAYAH,BLACKPINK,SQUARE ONE,2016-08-08,0.836,-3.282,124.969,4:00


In [71]:
def connect_to_db(host_name, dbname, port, username, password):
  
  try:
    conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

  except ps.OperationalError as e:
    raise e
  else:
    print('Connected!')
  
  return conn


def create_table(curr):
  create_table_command = ("""CREATE TABLE IF NOT EXISTS tracks (
                      track_id VARCHAR(255) PRIMARY KEY,
                      track_title TEXT NOT NULL,
                      track_artists TEXT NOT NULL,
                      track_album TEXT NOT NULL,
                      track_release_date TEXT NOT NULL,
                      energy FLOAT NOT NULL,
                      loudness FLOAT NOT NULL,
                      tempo FLOAT NOT NULL,
                      duration TEXT NOT NULL
              )""")
  
  curr.execute(create_table_command)


def insert_into_table(curr, track_id, track_title, track_artists, track_album, track_release_date, energy, loudness, tempo, duration):
  insert_into_tracks = ("""INSERT INTO tracks (track_id, track_title, track_artists,
                        track_album, track_release_date, energy, loudness, tempo, duration)
  VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s);""")
  row_to_insert = (track_id, track_title, track_artists, track_album, track_release_date, energy, loudness, tempo, duration)
  curr.execute(insert_into_tracks, row_to_insert)


def update_duration(curr, track_id, duration):
   query = ("""UPDATE tracks
            SET duration = %s
            WHERE track_id = %s;""")
   curr.execute(query, (duration,track_id))

def check_if_track_exists(curr, track_id): 
  query = ("""SELECT track_id FROM tracks WHERE track_id = %s""")

  curr.execute(query, (track_id,))
  return curr.fetchone() is not None


def append_from_df_to_db(curr, df):
  for i, row in df.iterrows():
      insert_into_table(curr, row['track_id'], row['track_title'], row['track_artists'], row['track_album'],
                        row['track_release_date'], row['energy'], row['loudness'], row['tempo'],
                        row['duration'])


def update_db(curr, df):
  tmp_df = pd.DataFrame(columns=['track_id', 'track_title', 'track_artists', 'track_album',
                                  'track_release_date', 'energy', 'loudness', 'tempo',
                                  'duration'])
  for i, row in df.iterrows():
      if check_if_track_exists(curr, row['track_id']):
        update_duration(curr, row['track_id'], row['duration'])
        continue
      else: # The track doesn't exist so we will add it to a temp df and append it using append_from_df_to_db
          tmp_df = tmp_df.append(row)

  return tmp_df

In [72]:
# main

# database credentials
host_name = '******.rds.amazonaws.com'
dbname = '******'
port = '5432'
username = '******' 
password = '******'
conn = None

# establish connection to db
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [73]:
# create table
create_table(curr)

In [74]:
# update db for new tracks
new_track_df = update_db(curr, df)
conn.commit()

# insert new tracks into db table
append_from_df_to_db(curr, new_track_df)
conn.commit()

In [75]:
# view data in db table
curr.execute("SELECT * FROM tracks WHERE (track_artists='Sum 41')")
print(curr.fetchall())

[('15O6qHmlCHmCbYtmeQ4W17', 'Hooch', 'Sum 41', 'Does This Look Infected?', '2002-01-01', 0.89, -5.784, 187.88099999999997, '3:28'), ('3O6ghD3ZG03dgCcdW4fYmV', "All She's Got", 'Sum 41', 'All Killer, No Filler', '2001-01-01', 0.958, -4.232, 120.07799999999999, '2:21'), ('4KtFCX32H4sBAIlRL4G96x', 'Dear Father', 'Sum 41', 'Underclass Hero', '2007-01-01', 0.885, -4.444, 131.996, '3:52'), ('7xuhVUJmBpHR276Yc7AsgW', 'Walking Disaster', 'Sum 41', 'Underclass Hero', '2007-01-01', 0.9390000000000001, -3.085, 111.236, '4:46'), ('1zqAXXA12UEVS7iDmAVeay', 'Screaming Bloody Murder', 'Sum 41', 'Screaming Bloody Murder', '2011-01-01', 0.987, -4.3, 189.99400000000003, '3:24'), ('6dXizHF3KbmdvOgvMAhnQC', 'Underclass Hero', 'Sum 41', 'Underclass Hero', '2007-01-01', 0.988, -2.9789999999999996, 191.933, '3:14'), ('74T57UIZY6gLdIahREmHbp', 'Billy Spleen', 'Sum 41', 'Does This Look Infected?', '2002-01-01', 0.8420000000000001, -5.282, 114.038, '2:32'), ('7FQW8qBMfmLD2jCV17DtI1', 'Back Where I Belong', 'Sum